In [69]:
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [70]:
SCRIPT_PATH = os.getcwd()
INPUT_DIR = os.path.abspath(os.path.join(SCRIPT_PATH, '..', 'data', 'preprocessed', 'spectrograms'))
OUTPUT_DIR = os.path.join(SCRIPT_PATH, '..', 'models')

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [71]:
# === Parameters ===
IMG_SIZE = (224, 224)   # wymagany przez VGG16
BATCH_SIZE = 16
SEED = 42

In [72]:
train_dir = os.path.join(INPUT_DIR, 'train')
val_dir = os.path.join(INPUT_DIR, 'val')
test_dir = os.path.join(INPUT_DIR, 'test')

In [73]:
train_ds = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    seed=SEED,
    shuffle=True
)

val_ds = image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    seed=SEED,
    shuffle=False
)

test_ds = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    seed=SEED,
    shuffle=False
)

Found 128 files belonging to 4 classes.
Found 24 files belonging to 4 classes.
Found 32 files belonging to 4 classes.


In [74]:
AUTOTUNE = tf.data.AUTOTUNE
def prepare(ds, training=False):
    ds = ds.map(lambda x,y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.shuffle(1024, seed=SEED)
    return ds.prefetch(AUTOTUNE)

train_ds = prepare(train_ds, training=True)
val_ds = prepare(val_ds, training=False)
test_ds = prepare(test_ds, training=False)

In [75]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.05),
    layers.RandomContrast(0.05),
], name="data_augmentation")

In [76]:
num_classes = 0

for batch in train_ds.take(1):
    _, y = batch
    num_classes = y.shape[-1]
print("Liczba klas:", num_classes)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))
base_model.trainable = False

model = models.Sequential([
    Input(shape=(224,224,3)),
    data_augmentation,
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
], name="vgg16_combined")

model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Liczba klas: 4


Model: "vgg16_combined"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,068 (56.64 MB)

 Trainable params: 132,868 (519.02 KB)

 Non-trainable params: 14,715,200 (56.13 MB)

In [80]:
# === CALLBACKS ===
callbacks = [
    #EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss'),
    ModelCheckpoint('best_vgg16_model.keras', save_best_only=True)
]

In [81]:
# === TRAIN THE MODEL ===
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=callbacks
)

Epoch 1/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 954ms/step - accuracy: 0.3125 - loss: 1.6091 - val_accuracy: 0.3333 - val_loss: 1.5580
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 940ms/step - accuracy: 0.2266 - loss: 1.7294 - val_accuracy: 0.2500 - val_loss: 1.5829
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 981ms/step - accuracy: 0.3125 - loss: 1.7286 - val_accuracy: 0.2500 - val_loss: 1.5811
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 950ms/step - accuracy: 0.3594 - loss: 1.6282 - val_accuracy: 0.2500 - val_loss: 1.5900
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 943ms/step - accuracy: 0.3750 - loss: 1.5931 - val_accuracy: 0.2500 - val_loss: 1.6230
Epoch 6/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 962ms/step - accuracy: 0.3906 - loss: 1.4965 - val_accuracy: 0.2500 - val_loss: 1.6476
Epoch 7/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.2891 - loss: 1.7366 - val_accuracy: 0.2500 - val_loss: 1.6583
Epoch 8/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 913ms/step - accuracy: 0.4141 - loss: 1.4482 - val_accuracy: 0.2500 - val_loss: 1.

In [82]:
# === EVALUATE THE MODEL ===
test_loss, test_acc = model.evaluate(test_ds)
print(f"\n Test accuracy: {test_acc*100:.2f}%")
model.save(os.path.join(OUTPUT_DIR,'vgg16_motor_fault_classifier_final.keras'))
print("\n💾 Model saved as: vgg16_motor_fault_classifier_final.keras")

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 814ms/step - accuracy: 0.2500 - loss: 2.0620

 Test accuracy: 25.00%

💾 Model saved as: vgg16_motor_fault_classifier_final.keras
